# Model.ipynb
This file contains all the necessary features to build our model from scratch:
- Generating and Saving Data
- Loading the Data
- Building the Model Structure
- Compiling the Model
- Fitting/Training the Model
- Saving the Model


## Generate Data

In [ ]:
import chess
import chess.engine
import random
import numpy as np
import os

# This creates randoms boards we will feed to our model
def gen_random_board(max_moves=200):
    board = chess.Board()
    depth = random.randrange(0, max_moves)

    for _ in range(depth):
        all_possible_moves = list(board.legal_moves)
        move = random.choice(all_possible_moves)
        board.push(move)
        if board.is_game_over():
            break
    
    return board

# This returns the Stockfish eval which we will feed to our model
def stockfish(board, depth):
    with chess.engine.SimpleEngine.popen_uci("/usr/games/stockfish") as sf:
        result = sf.analyse(board, chess.engine.Limit(depth=depth))
        score = result['score'].white().score()
        return score

In [ ]:
board = gen_random_board()
board

In [ ]:
print(stockfish(board, 10))

In [ ]:
squares_index = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def convert_to_3d(board):
    board3d = np.zeros((14,8,8), dtype=np.int8)

    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8,8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8,8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1

    aux = board.turn
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i, j, = square_to_index(move.to_square)
        board3d[12][i][j] = 1
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i, j = square_to_index(move.to_square)
        board3d[13][i][j] = 1
    board.turn = aux
    
    return board3d

In [ ]:
boards = np.arange(89600000, dtype=np.int8).reshape(100000, 14, 8, 8)
evals = np.arange(100000, dtype=np.int32)

for i in range(100000):
    board = gen_random_board()
    eval = stockfish(board, 10)

    eval = eval if eval else 0
    
    boards[i] = convert_to_3d(board)
    evals[i] = eval

file_name = 'data.npz'
np.savez(os.path.join('data', file_name), b=boards, v=evals)

## Load Data

In [5]:
import numpy as np
import os

file_name = 'data.npz'  # Change this to use a different dataset from the data folder

def get_dataset():
    data = np.load(os.path.join('data', file_name))
    b, v = data['b'], data['v']
    v = np.asarray( v / abs(v).max() / 2 + 0.5, dtype=np.float32)

    return b, v

x_train, y_train = get_dataset()
y_train.resize(100000)

def get_test_dataset():
    data = np.load(os.path.join('data', file_name))
    b, v = data['b'], data['v']
    v = np.asarray( v / abs(v).max() / 2 + 0.5, dtype=np.float32)

    return b, v

x_test, y_test = get_test_dataset()
y_test.resize(100000)



## AI Convolution Neural Network Model

### Build Model

In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import tensorflow.keras.layers as layers

inputs = layers.Input(shape=(14, 8, 8))

x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(inputs)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.sigmoid)(x)

x = layers.Flatten()(x)
x = layers.Dense(32, tf.nn.sigmoid)(x)

outputs = layers.Dense(1, tf.nn.sigmoid)(x)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 14, 8, 8)]        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 8, 32)         2336      
                                                                 
 batch_normalization_4 (Bat  (None, 14, 8, 32)         128       
 chNormalization)                                                
                                                                 
 conv2d_6 (Conv2D)           (None, 14, 8, 32)         9248      
                                                                 
 batch_normalization_5 (Bat  (None, 14, 8, 32)         128       
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 8, 64)         1849

### Compile Model

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=tf.keras.losses.MeanSquaredError())

### Fit/Train Model

In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import datetime

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

log_name = 'Chess-CNN-Model'
log_dir = os.path.join('logs', log_name)
tb = TensorBoard(log_dir=log_dir) 

checkpoint_filepath = '/tmp/checkpoint/'

callbacks = [
    tb,
    
    ModelCheckpoint(
        filepath = checkpoint_filepath,
        save_weights_only = False,
        save_best_only = True,
    )
]

model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1, validation_split=0.1, callbacks=callbacks, validation_data=(x_test, y_test), validation_batch_size=32)


Epoch 1/10
3118/3125 [============================>.] - ETA: 0s - loss: 1.6711e-04INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


3125/3125 [==============================] - 23s 6ms/step - loss: 1.6705e-04 - val_loss: 1.1874e-04
Epoch 2/10
3114/3125 [============================>.] - ETA: 0s - loss: 1.1639e-04INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


3125/3125 [==============================] - 18s 6ms/step - loss: 1.1637e-04 - val_loss: 1.0836e-04
Epoch 3/10
3120/3125 [============================>.] - ETA: 0s - loss: 1.0718e-04INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


...

3125/3125 [==============================] - 19s 6ms/step - loss: 9.3249e-05 - val_loss: 9.2233e-05
Epoch 9/10
3114/3125 [============================>.] - ETA: 0s - loss: 9.1634e-05INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


3125/3125 [==============================] - 19s 6ms/step - loss: 9.1828e-05 - val_loss: 8.9276e-05
Epoch 10/10
3117/3125 [============================>.] - ETA: 0s - loss: 9.0523e-05INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


INFO:tensorflow:Assets written to: /tmp/checkpoint/assets


3125/3125 [==============================] - 19s 6ms/step - loss: 9.0530e-05 - val_loss: 8.8839e-05


### Save Model
- If we like the performance of our model, we should save it
- The following line of code saves our model to a '.keras' file
    - We can load this back in and train it more
    - Or we can load it into our chess game and play against it

In [9]:
model.save('models/model.keras')